## **Etapa 1: Obtencion de datos**

En esta práctica obtendrás los datos para predecir si un falcon 9 aterizará con éxito o no. Los datos serán recogidos mediante la API de SpaceX y nos aseguraremos de que siguen un formato adecuado para las fases siguientes. El siguiente es un ejemplo de un aterrizaje exitoso:

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing\_1.gif)


Aquí puedes ver varios ejemplos de aterrizajes fallidos:




![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


La gran mayoría de aterrizajes fallidos son intencionados para la realización de diversos controles. Estos aterrizajes se llevan a cabo la mayoria en el oceano, veremos estadísticas al respecto.

## Objetivos



En esta práctica realizaras una GET request a la API de SpaceX. Tambien realizarás cierto data wrangling básico y limpieza de datos.

***


## Importación de librerias.

Nos importamos las siguientes librerías necesarias para el desarrollo de la práctica.

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#Las opciones anteriores aseguran que pandas muestre el df completo en jupyternotebooks

Realicemos ahora una petición GET a la API de SpaceX. La URL es la siguiente.


In [2]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

* Usando la librería requests usa el método GET y pasale como argumento URL anterior, guardala en una variable `response`  y realiza el print de `response.content`

In [ ]:
#INSERTA AQUÍ TU CÓDIGO
#requests.TIPODEPETICIÓN(URL)

response = requests.get('https://api.spacexdata.com/v4/launches/past')
print(response.content)

Si la petición ha sido realizada correctamente el resultado del print debería ser similar al siguiente:


```
b'[{"fairings":{"reused":false,"recovery_attempt":false,"recovered":false, ...
```
(Recomendamos cerrar el resultado del print después de revisarlo para que google collab no sufra.)

Nuestra variable `response` contiene una grandísima cantidad de información sobre los SpaceX pero vamos a intentar formatearla para poder ver de que se trata exactamente y que nos interesa.


### Parte 1: Obtén los datos mediante peticiones GET y crea un dataframe

Hemos observado que la variable `response` tiene forma de diccionario (como la mayoría de respuestas a peticiones GET). Una forma efectiva de formatear dichos datos es transformarlos en un JSON y posteriormente en un dataframe de python:


In [5]:
data=response.json()
data=pd.json_normalize(data)

*Utilizando el dataframe anterior muestra las 5 primeras filas*

In [ ]:
#INSERTA AQUÍ TU CÓDIGO
data[:6]

Observemos que la mayoría de datos interesantes no se muestran de forma explícita sino que son IDs. Estos IDs nos permiten obtener mas información haciendo pediciones GET a diferentes endpoints de la API.
Para mas infrmación: https://docs.spacexdata.com/

Revisando la documentación de la API nos hemos dado cuenta de que no son necesarias todas las columnas, por lo que vamos a reducir nuestro dataframe a lo crucial:

In [11]:
#Reescribe el dataframe para quedarnos solo con las columnas'rocket', 'payloads','success', 'launchpad', 'cores', 'flight_number', 'date_utc'
#INSERTA AQUÍ TU CÓDIGO:

data=data[['rocket','payloads','success','launchpad','cores','flight_number','date_utc']]

In [12]:
# Algunos cohetes tienen mas de una carga 'payload' o 'core'.
# Estos datos nos darán problemas en un futuro, entonces nos limitearemos a aquellos que solamente tienen un core o una carga (payload):
data = data[data['cores'].map(len)==1]
#INSERTA AQUI TU CÓDIGO para payloads
data = data[data['payloads'].map(len)==1]

In [13]:
# Otro problema que tenemos es que ahora tenemos listas de longitud 1.
# Las listas de python no son especialmente compatibles con SQL por lo que lo solucionaremos sacando el valor de la lista:
data['cores'] = data['cores'].map(lambda x : x[0])
#INSERTA AQUI TU CÓDIGO para hacer lo mismo con la columna payloads
data['payloads'] = data['payloads'].map(lambda x : x[0])

In [14]:
# El formato de la fecha no es especialmete cómodo por lo que lo formatearemos:
data['date'] = pd.to_datetime(data['date_utc']).dt.date

No se te olvide ejecutar la celda anterior (1 sola vez)

-------------------

Si revisamos `rocket, payload,launchpad y cores` son IDs, estos IDs nos permiten hacer peticiones a diferetes endpoints y obtener más información. La información para cada variable se encuentra en los siguientes links:

*   Para <code>rocket</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/rockets/v4/one.md . De aquí nos quedaremos la versión del cohete propulsor.

*   Para <code>payload</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/payloads/v4/one.md . De aquí guardaremos la carga, la masa de esta carga, la orbita a la que fue enviada y el cliente.

*   Para <code>launchpad</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/launchpads/v4/one.md . Guardamos la latitud, longitud y nombre de las diferentes plataformas de lanzamiento.

*   Para <code>cores</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/cores/v4/one.md . Gaurdamos diferentes variables del nucleo.

Los datos serán guardados en listas y estas listas las utilizaremos para crear una nuevo dataframe con todos los datos:



In [15]:
#Para rocket
BoosterVersion = []

#Para paylaod
PayloadMass = []
Payload = []
Orbit = []
Customers= []

#Para launchpad
LaunchSite = []
Longitude = []
Latitude = []

#Para core
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
Mission_Outcome = []
ReusedCount = []
Serial = []

Para completar las listas anteriores nos definiremos una lista de funciones para facilitar la obtención de los datos.

Empecemos con los cohetes:

In [38]:
# DEFINICIÓN FUNCIÓN 1
def getBoosterVersion(data): #Definimos la función
    for x in data['rocket']: #Iteramos por cada fila de la columna rocket
       if x: #Comprobamos que no este vacía
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)) #Realizamos la petición GET (Aplicamos str() para poder concatenar)
        response = response.json() #Transformamos la respuesta en un diccionario.
        BoosterVersion.append(response['name']) #Añadimos a BoosterVersion el valor con key 'name'

De <code>launchpad</code> nos gustaría quedarnos con la longitud, la latitud y el nombre de la plataforma


In [29]:
# DEFINICIÓN FUNCIÓN 2
#INSERTA AQUI TU CÓDIGO
def getLaunchSite(data): #Definimos la función
    for x in data['launchpad']: #Iteramos por cada fila de la columna rocket
       if x: #Comprobamos que no este vacía
        response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)) #Realizamos la petición GET (Aplicamos str() para poder concatenar)
        response = response.json() #Transformamos la respuesta en un diccionario.
        Longitude.append(response['longitude']) #Añadimos a Longitude el valor con key 'longitude'
        Latitude.append(response['latitude']) #Añadimos a Latitude el valor con key 'latitude'
        LaunchSite.append(response['name']) #Añadimos a LaunchSite el valor con key 'name'


De <code>payload</code> nos gustaría obtener la carga, el peso de la carga, el cliente y la órbita a la que fué enviada.

In [47]:
# DEFINICIÓN FUNCIÓN 3
#INSERTA AQUI TU CÓDIGO

def getPayloadData(data): #Definimos la función
    for x in data['payloads']: #Iteramos por cada fila de la columna rocket
       if x: #Comprobamos que no este vacía
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+str(x)) #Realizamos la petición GET (Aplicamos str() para poder concatenar)
        response = response.json() #Transformamos la respuesta en un diccionario.
        Payload.append(response['name']) #Añadimos a Payload el valor con key 'name'
        Customers.append(response['customers']) #Añadimos a Customers el valor con key 'customers'
        PayloadMass.append(response['mass_kg']) #Añadimos a PayloadMass el valor con key 'mass_kg'
        Orbit.append(response['orbit']) #Añadimos a Orbit el valor con key 'orbit'


La función para completar los datos de `core` tiene una complicación extra por lo que la damos hecha:

In [31]:
# DEFINICIÓN FUNCIÓN 4
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])


Las funciones anteriores rellenan las listas vacías, veamoslo en práctica. *Revisa que la lista BoosterVersion esta vacía*


In [39]:
#INSERTA AQUI TU CÓDIGO
if not BoosterVersion:
    print("La lista BoosterVersion está vacía")
else:
    print("La lista BoosterVersion no está vacía")

La lista BoosterVersion está vacía


*Llama ahora la función `getBoosterVersion`.* (No devuelve nada así que no hace falta asignarle una variable. Solo instanciamos la funcion.)

In [41]:
# LLAMADA A FUNCIÓN 1
#INSERTA AQUI TU CÓDIGO
getBoosterVersion(data)

*Muestra los primeros 5 valores de la lista `BoosterVersion`*

In [43]:
#INSERTA AQUI TU CÓDIGO
BoosterVersion[:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

*Aplica el resto de funciones:*
(No devuelven nada así que no hace falta asignarles una variable. Solo instanciamos a las funciones.)

In [44]:
# LLAMADA A FUNCIÓN 2
#INSERTA AQUI TU CÓDIGO
getLaunchSite(data)

In [48]:
# LLAMADA A FUNCIÓN 3
#INSERTA AQUI TU CÓDIGO
getPayloadData(data)

In [49]:
# LLAMADA A FUNCIÓN 4
#INSERTA AQUI TU CÓDIGO
getCoreData(data)

Finalmente creamos un dataframe desde las listas anteriores. Primero convirtámolos en un diccionario:

In [50]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Payload':Payload,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'LandingOutcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'Customers': Customers,
'Mission_Outcome':list(data['success']),
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

Y ahora crea un dataframe de Pandas desde el diccionario. Llámalo `launch_data`

In [57]:
#INSERTA AQUI TU CÓDIGO
launch_data = pd.DataFrame(launch_dict)

Finalmente, muestra las primeras 5 filas del dataframe.

In [58]:
#INSERTA AQUI TU CÓDIGO
launch_data[:5:]

,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,FalconSAT-2,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[DARPA],False,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,DemoSAT,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[DARPA],False,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,RatSat,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[SpaceX],True,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,RazakSAT,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[ATSB],True,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857


### Parte 2: Filtra el dataframe para que solo incluya los lanzamientos de falcon 9



El falcon 9 es una versión considerablemente más nueva y efectiva que el faclon 1. Dado que nos interesan los datos recientes y a futuro vamos a *eliminar los lanzamientos de `Falcon 1` del fataframe launch_data y llama al nuevo dataframe `data_falcon9`. Muestra las primeras 5 filas.*

In [61]:
# INSERTA AQUÍ TU CÓDIGO
data_falcon9 = launch_data[launch_data['BoosterVersion'] == 'Falcon 9']


Ahora que hemos eliminado los lanzamientos de Falcon 1 la columna FlightNumber está desajustada. Vamos a volver a ordenarla:

In [62]:
data_falcon9.FlightNumber = list(range(1, data_falcon9.shape[0]+1))
data_falcon9.head()

<ipython-input-62-5e3e7c2c50f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9.FlightNumber = list(range(1, data_falcon9.shape[0]+1))


,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,COTS Demo Flight 2,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA(COTS)],True,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,CRS-2,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA (CRS)],True,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,CASSIOPE,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,[MDA],True,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,SES-8,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SES],True,0,B1004,-80.577366,28.561857


### Parte 3: Ajustes finales


Debajo podemos observar que nos faltan ciertos valores.

In [65]:
data_falcon9.isnull().sum()

FlightNumber        0
Date                0
BoosterVersion      0
PayloadMass        22
Payload             0
Orbit               1
LaunchSite          0
LandingOutcome      0
Flights             0
GridFins            0
Reused              0
Legs                0
LandingPad         26
Block               0
Customers           0
Mission_Outcome     1
ReusedCount         0
Serial              0
Longitude           0
Latitude            0
dtype: int64

* Lo óptimo sería no tener celdas vacías por lo que vamos a ver que podemos hacer para solucionar esta situación.
La columna <code>LandingPad</code> tiene valores `None`en los casos donde el cohete no tuvo intención de aterrizar por lo que no tenía ningún LandingPad asignado.

* Por otro lado, los valores innexistentes de `PayloadMass` si se pueden completar. *En este caso utilizaras la función `.mean()` para calcular la media y `.replace()` para reemplzar los valores `np.nan`.*


In [69]:
# INSERTA AQUI TU CÓDIGO
payload_mass_mean = data_falcon9['PayloadMass'].mean()
data_falcon9['PayloadMass'].replace(np.nan, payload_mass_mean, inplace=True)

data_falcon9.isnull().sum()

<ipython-input-69-b62f3e3fa5d4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9['PayloadMass'].replace(np.nan, payload_mass_mean, inplace=True)


FlightNumber        0
Date                0
BoosterVersion      0
PayloadMass         0
Payload             0
Orbit               1
LaunchSite          0
LandingOutcome      0
Flights             0
GridFins            0
Reused              0
Legs                0
LandingPad         26
Block               0
Customers           0
Mission_Outcome     1
ReusedCount         0
Serial              0
Longitude           0
Latitude            0
dtype: int64

El número de celdas vacías en la columna <code>PayLoadMass</code> debería cambiar a 0.


Por último tenemos que la columna `Customers` es una lista, y como hemos comentado previamente, no se llevan especialmente bien con SQL.
*Transforma esta lista en una string aplicando `.map(lambda x : ','.join(x))` a la columna y guardalo en la misma columna.*

In [70]:
#INSERTA AQUÍ TU CÓDIGO
data_falcon9['Customers'].map(lambda x : ','.join(x))

4             SpaceX
5         NASA(COTS)
6         NASA (CRS)
7                MDA
8                SES
           ...      
167           SpaceX
168           SpaceX
169           SpaceX
170           SpaceX
171    NASA (CCtCap)
Name: Customers, Length: 168, dtype: object

*Ahora exporta el dataset en formato `.csv`  con `index=False` y llámalo `dataset_part_1.csv`. Descarga el archivo porque será necesario en la siguiente práctica.*


In [71]:
#INSERTA AQUÍ TU CÓDIGO
data_falcon9.to_csv("dataset_part_1.csv",index=False)